## Higher Fidelity Graph States
In this notebook, we will prepare quantum circuits for a 7-qubit graph state and estimate the fidelity through stabilizer measurements using Qiskit. In this example, we use the CTMP method of error mitigation and repeat the graph state measurement 16 times to find statistical error bars.

**To Do:
Modify the graph state preparation circuit or use your own methods of error mitigation to improve the graph state fidelity.**

### Our Solution:
We focus on improving gate fidelities by manipulating the graph state function to minimize decoherence. We modify the way the circuit gates run to optimize and reduce noise error. We calibrate our measurements for the CTMP error mitigation using quantum states with two-qubit excitations with custom parameters.

## Imports

Begin by importing the necessary packages and defining the functions we will need for the stabilizer measurements.

In [ ]:
### install Qiskit and other modules if you don't have them already
!pip install -r dependencies/requirements.txt --quiet

In [ ]:
# Qiskit module
!pip install qiskit
from qiskit import QuantumCircuit
import qiskit.circuit.library as circuit_library
import qiskit.quantum_info as qi
import qiskit.ignis.mitigation as mit

# Qiskit tools for noisy simulation
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.utils import insert_noise

# Qiskit tools for running and monitoring jobs
from qiskit import execute
from qiskit.tools.monitor import job_monitor

# Other imports
import numpy as np

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In order to run the circuits, first load the backend `ibmq_casablanca` from your account using the `IBMQ` provider. You will receive access to `ibm-q-community/ibmquantumawards/open-science` after registering for the Open Science Prize.

In [ ]:
# Load IBMQ Account data
from qiskit import IBMQ
IBMQ.load_account()

# Get backend for experiment
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science')
backend = provider.get_backend('ibmq_casablanca')
properties = backend.properties()

## Preparing graph states

Here, we prepare the graph state circuit for 7-qubits using the `GraphState` function in Qiskit's circuit library. We define a graph that uses the connectivity map of the quantum system `ibmq_casablanca`.

In [1]:

num_qubits = 7

# adjacency matrix for `ibmq_casablanca`
adjmat = [
    [0, 1, 0, 0, 0, 0, 0], 
    [1, 0, 1, 1, 0, 0, 0], 
    [0, 1, 0, 0, 0, 0, 0], 
    [0, 1, 0, 0, 0, 1, 0], 
    [0, 0, 0, 0, 0, 1, 0], 
    [0, 0, 0, 1, 1, 0, 1], 
    [0, 0, 0, 0, 0, 1, 0]]

### Your code goes here

How would you prepare a graph state with high fidelity? In the example below, we create it using Qiskit's circuit library at the gate level. You may explore other methods for creating the graph states, including by using pulse-level techniques or accounting for the errors in the quantum system.

In [ ]:
from typing import Union, List
import math
class GraphStateKP(QuantumCircuit):
    def __init__(self,
                 adjacency_matrix: Union[List, np.array]) -> None:
        """Create graph state preparation circuit.

        Args:
            adjacency_matrix: input graph as n-by-n list of 0-1 lists

        Raises:
            CircuitError: If adjacency_matrix is not symmetric.

        The circuit prepares a graph state with the given adjacency
        matrix.
        """

        adjacency_matrix = np.asarray(adjacency_matrix)
        if not np.allclose(adjacency_matrix, adjacency_matrix.transpose()):
            raise CircuitError("The adjacency matrix must be symmetric.")

        num_qubits = len(adjacency_matrix)
        super().__init__(num_qubits, name="graph: %s" % (adjacency_matrix))
        self.h(0)
        self.sdg(1)
        self.h(2)
        self.sdg(3)
        self.h(4)
        self.sdg(5)
        self.h(6)
        self.cx(0,1)
        self.cx(2,3)
        self.cx(4,5)
        self.s(1)
        self.s(3)
        self.s(5)
        self.sdg(6)
        self.cx(4,6)
        
def create_graph_state():
    graph_state_circuit = GraphStateKP(adjmat)
    return graph_state_circuit

In [ ]:
# the graph state can be created using Qiskit's circuit library
state_circuit = create_graph_state()
state_circuit.draw()

q_0: ──────────■─────────────
     ┌─────┐ ┌─┴─┐ ┌───┐     
q_1: ┤ SDG ├─┤ X ├─┤ S ├─────
     └┬───┬┘ └───┘ └───┘     
q_2: ─┤ H ├────■─────────────
     ┌┴───┴┐ ┌─┴─┐ ┌───┐     
q_3: ┤ SDG ├─┤ X ├─┤ S ├─────
     └┬───┬┘ └───┘ └───┘     
q_4: ─┤ H ├────■──────────■──
     ┌┴───┴┐ ┌─┴─┐ ┌───┐  │  
q_5: ┤ SDG ├─┤ X ├─┤ S ├──┼──
     └┬───┬┘┌┴───┴┐└───┘┌─┴─┐
q_6: ─┤ H ├─┤ SDG ├─────┤ X ├
      └───┘ └─────┘     └───┘

## Creating and measuring stabilizers

We begin by defining functions to create stabilizer measurement circuits, and then appending them onto the circuit used to create the graph states.

In [ ]:
def compute_stabilizer_group(circuit):
    """Compute the stabilizer group for stabilizer circuit."""
    state = qi.Statevector.from_instruction(circuit)
    labels = []
    for i in qi.pauli_basis(state.num_qubits):
        val = round(qi.state_fidelity(i.to_matrix()[0], state, validate=False))
        if val != 0:
            label = i.to_labels()[0]
            if val == 1:
                label = '+' + label
            else:
                label = '-' + label
            labels.append(label)
    return labels

def stabilizer_coeff_pauli(stabilizer):
    """Return the 1 or -1 coeff and Pauli label."""
    coeff = 1
    pauli = coeff
    if stabilizer[0] == '-':
        coeff = -1
    if stabilizer[0] in ['+', '-']:
        pauli = stabilizer[1:]
    else:
        pauli = stabilizer
    return coeff, pauli

def stabilizer_measure_circuit(stabilizer, initial_circuit=None):
    """Return a stabilizer measurement circuits.
    
    Args:
        stabilizer (str): a stabilizer string
        initial_circuit (QuantumCircuit): Optional, the initial circuit.
    
    Returns:
        QuantumCircuit: the circuit with stabilizer measurements.
    """
    _, pauli = stabilizer_coeff_pauli(stabilizer)
    if initial_circuit is None:
        circ = QuantumCircuit(len(pauli))
    else:
        circ = initial_circuit.copy()
    for i, s in enumerate(reversed(pauli)):
        if s == 'X':
            circ.h(i)
        if s == 'Y':
            circ.sdg(i)
            circ.h(i)
    circ.measure_all()
    return circ

In [ ]:
## Compute the stabilizers for this graph state
generators = qi.Clifford(state_circuit).stabilizer.pauli.to_labels()
stabilizers = compute_stabilizer_group(state_circuit)
print('Stabilizers:', stabilizers)
print('Generators:', generators)

Stabilizers: ['+IIIIIII', '+IIIIIIZ', '+IIIIIZI', '+IIIIIZZ', '+IIIXYII', '+IIIXYIZ', '+IIIXYZI', '+IIIXYZZ', '+IIIYXII', '+IIIYXIZ', '+IIIYXZI', '+IIIYXZZ', '+IIIZZII', '+IIIZZIZ', '+IIIZZZI', '+IIIZZZZ', '+IZZIIII', '+IZZIIIZ', '+IZZIIZI', '+IZZIIZZ', '+IZZXYII', '+IZZXYIZ', '+IZZXYZI', '+IZZXYZZ', '+IZZYXII', '+IZZYXIZ', '+IZZYXZI', '+IZZYXZZ', '+IZZZZII', '+IZZZZIZ', '+IZZZZZI', '+IZZZZZZ', '+XXYIIII', '+XXYIIIZ', '+XXYIIZI', '+XXYIIZZ', '+XXYXYII', '+XXYXYIZ', '+XXYXYZI', '+XXYXYZZ', '+XXYYXII', '+XXYYXIZ', '+XXYYXZI', '+XXYYXZZ', '+XXYZZII', '+XXYZZIZ', '+XXYZZZI', '+XXYZZZZ', '+XYXIIII', '+XYXIIIZ', '+XYXIIZI', '+XYXIIZZ', '+XYXXYII', '+XYXXYIZ', '+XYXXYZI', '+XYXXYZZ', '+XYXYXII', '+XYXYXIZ', '+XYXYXZI', '+XYXYXZZ', '+XYXZZII', '+XYXZZIZ', '+XYXZZZI', '+XYXZZZZ', '-YIZIIII', '-YIZIIIZ', '-YIZIIZI', '-YIZIIZZ', '-YIZXYII', '-YIZXYIZ', '-YIZXYZI', '-YIZXYZZ', '-YIZYXII', '-YIZYXIZ', '-YIZYXZI', '-YIZYXZZ', '-YIZZZII', '-YIZZZIZ', '-YIZZZZI', '-YIZZZZZ', '-YZIIIII', '-YZIIIIZ', '-

In [ ]:
## Append the stabilizer measurements to the graph state circuit 
stabilizer_circuits = [stabilizer_measure_circuit(stab, state_circuit)
                       for stab in stabilizers]

stabilizer_circuits[0].draw()

░ ┌─┐                  
   q_0: ──────────■──────────────░─┤M├──────────────────
        ┌─────┐ ┌─┴─┐ ┌───┐      ░ └╥┘┌─┐               
   q_1: ┤ SDG ├─┤ X ├─┤ S ├──────░──╫─┤M├───────────────
        └┬───┬┘ └───┘ └───┘      ░  ║ └╥┘┌─┐            
   q_2: ─┤ H ├────■──────────────░──╫──╫─┤M├────────────
        ┌┴───┴┐ ┌─┴─┐ ┌───┐      ░  ║  ║ └╥┘┌─┐         
   q_3: ┤ SDG ├─┤ X ├─┤ S ├──────░──╫──╫──╫─┤M├─────────
        └┬───┬┘ └───┘ └───┘      ░  ║  ║  ║ └╥┘┌─┐      
   q_4: ─┤ H ├────■──────────■───░──╫──╫──╫──╫─┤M├──────
        ┌┴───┴┐ ┌─┴─┐ ┌───┐  │   ░  ║  ║  ║  ║ └╥┘┌─┐   
   q_5: ┤ SDG ├─┤ X ├─┤ S ├──┼───░──╫──╫──╫──╫──╫─┤M├───
        └┬───┬┘┌┴───┴┐└───┘┌─┴─┐ ░  ║  ║  ║  ║  ║ └╥┘┌─┐
   q_6: ─┤ H ├─┤ SDG ├─────┤ X ├─░──╫──╫──╫──╫──╫──╫─┤M├
         └───┘ └─────┘     └───┘ ░  ║  ║  ║  ║  ║  ║ └╥┘
meas: 7/════════════════════════════╩══╩══╩══╩══╩══╩══╩═
                                    0  1  2  3  4  5  6

## Measurement mitigation

Noisy measurements impact our ability to accurately measure the state fidelity. For our default example we calibrate our measurements for the CTMP method using states with two-qubit excitations.

In [ ]:
labels = ['0000000', '0000011', '0000101', '0000110',
          '0001001', '0001010', '0001100', 
          '0010001', '0010010', '0010100', '0011000', 
          '0100001', '0100010', '0100100', '0101000', '0110000', 
          '1000001', '1000010', '1000100', '1001000', '1010000', '1100000', 
          '1111111']
meas_cal_circuits, metadata = mit.expval_meas_mitigator_circuits(num_qubits, method='CTMP',labels=labels)

In [ ]:
[meas_cal_circuits_full, state_labels] = mit.complete_meas_cal(range(num_qubits))

## Run the circuits

We will run the circuits on the `ibmq_casablanca` quantum system.

In order to debug more quickly and avoid queues, you may consider using a simulator backend modeled after the real quantum system. This will use the noise model of `ibmq_casablanca` to do simulations. You may uncomment the line below to do so. **Note that the fidelities of your graph states will generally be higher on the simulator, but the Open Science Prize is awarded for the best fidelities on the real quantum system.**

In [ ]:
backend = QasmSimulator.from_backend(provider.get_backend('ibmq_casablanca'))

We run the measurement calibration circuits in a separate job from the graph state circuits.  We repeat both 16 times and use the mean as the final value. 

In order to debug more quickly, you may consider reducing `reps` from 16 to 1. **Note that the final submissions will need to be executed with 16 repetitions.**

In [ ]:
reps = 16

In [ ]:
all_jobs = []
all_jobs_mit = []

for ii in range(reps):

    # Run QPT on backend
    shots = 8192
    il = [0,1,2,3,4,5,6]
    
    job_backend = execute(stabilizer_circuits, backend, shots=shots, initial_layout=il)
    job_mit_backend = execute(meas_cal_circuits, backend, shots=shots, initial_layout=il)
    print('Job IDs ({}/{}): \n measurement calibration: {}\n stabilizer measurements: {}'.format(
        ii+1, reps, job_mit_backend.job_id(), job_backend.job_id()))

    all_jobs.append(job_backend)
    all_jobs_mit.append(job_mit_backend)

Job IDs (1/16): 
 measurement calibration: ac8239b3-640d-451e-ac30-ad5be7d7f873
 stabilizer measurements: 739fff17-2b54-493d-a567-e56427f9e334
Job IDs (2/16): 
 measurement calibration: f9ca633e-a3f7-429e-87d9-bc5075861c51
 stabilizer measurements: 8271568e-2255-4b4f-a48d-32f5857d913a
Job IDs (3/16): 
 measurement calibration: f34be18a-8cd4-411e-8f12-6bd286bc8a84
 stabilizer measurements: dad2a1bb-4704-42a0-8be1-c02a82a62b39
Job IDs (4/16): 
 measurement calibration: 0758bb50-f135-4a15-bd94-14916991f9f8
 stabilizer measurements: fbc3b79c-3d44-4757-8fc9-73cdfe49b617
Job IDs (5/16): 
 measurement calibration: 1f8cea35-3b7e-40aa-b1f6-348f4da79414
 stabilizer measurements: a24c2d81-f0aa-4ca8-a4ac-bd0d90d8d82d
Job IDs (6/16): 
 measurement calibration: bfcbd114-3399-41b7-a442-82f12fba4b88
 stabilizer measurements: 9401e811-a057-4537-bf4b-543fd11a7d02
Job IDs (7/16): 
 measurement calibration: c9f428aa-8c19-4d02-9380-c4a5caad2e1f
 stabilizer measurements: ea2a0de9-7f3c-4ebe-ab30-c7a92bf7344c

We can monitor the status of the jobs using Qiskit's job monitoring tools.

In [ ]:
for job in all_jobs:
    job_monitor(job)
    try:
        if job.error_message() is not None:
            print(job.error_message())
    except:
        pass

Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run


## Post-processing and computing fidelities

Once the jobs are completed, we can get the results back as follows.

In [ ]:
result_backend = []
result_mit_backend = []
for job in all_jobs:
    # Retrieve results (this may take a while depending on the queue)
    result_backend.append(job.result())
    
for job in all_jobs_mit:
    result_mit_backend.append(job.result())

Finally, we compute the fidelities of the graph states. You may consider creating your own method for error mitigation by updating the `stabilizer_expvals` function below. Here, we will use the default methods provided in Qiskit.

In [ ]:
def stabilizer_measure_diagonal(stabilizer):
    """Return the diagonal vector for a stabilizer measurement.
    
    Args:
        stabilizer (str): a stabilizer string
    
    Returns:
        np.ndarray: the diagonal for measurement in the stabilizer basis.
    """
    coeff, pauli = stabilizer_coeff_pauli(stabilizer)
    diag = np.array([1])
    for s in reversed(pauli):
        if s == 'I':
            tmp = np.array([1, 1])
        else:
            tmp = np.array([1, -1])
        diag = np.kron(tmp, diag)
    return coeff * diag
    
def stabilizer_fidelity(expvals, stddevs=None):
    """Compute stabilizer state fidelity from stabilizer expvals."""
    mean = np.mean(expvals)
    if stddevs is None:
        return mean
    stddev = np.sqrt(np.sum(stddevs ** 2))
    return mean, stddev

### Your code goes here

You may consider updating the function below to change how the measurement calibration circuits are used to compute the fidelity of the graph state.

In [ ]:
def stabilizer_expvals(result, stabilizers, meas_mitigator=None):
    """Compute expectation values from stabilizer measurement results."""
    
    expvals = []
    stddevs = []
    for i, stab in enumerate(stabilizers):
        expval, stddev = mit.expectation_value(
            result.get_counts(i),
            diagonal=stabilizer_measure_diagonal(stab),
            meas_mitigator=meas_mitigator)
        expvals.append(expval)
        stddevs.append(stddev)
    return np.array(expvals), np.array(stddevs)


In [ ]:
## Mitigate the stabilizer expectation values 
F_nomit_backend = []
F_mit_backend = []

for ii in range(reps):
    # Unmitigated Expectation Values
    expvals_nomit_b, stddevs_nomit_b = stabilizer_expvals(
        result_backend[ii], stabilizers)
  
    # Fit measurement error mitigators
    mitigator_backend = mit.ExpvalMeasMitigatorFitter(result_mit_backend[ii], metadata).fit()

    # Measurement error mitigated expectation values
    expvals_mit_b, stddevs_mit_b = stabilizer_expvals(
        result_backend[ii], stabilizers, meas_mitigator=mitigator_backend)
    
    # save the fidelities for this iteration
    F_nomit_backend.append(stabilizer_fidelity(expvals_nomit_b, stddevs_nomit_b)[0])
    F_mit_backend.append(stabilizer_fidelity(expvals_mit_b, stddevs_mit_b)[0])

Report the fidelity estimates.

In [ ]:
## The final results

print('Graph-state fidelity estimates')
print('\nNo mitigation')
print('F({}) = {:.3f} \u00B1 {:.3f}'.format(
    properties.backend_name, np.mean(F_nomit_backend), np.std(F_nomit_backend)))

print('\nCTMP error mitigation')
print('F({}) = {:.3f} \u00B1 {:.3f}'.format(
    properties.backend_name, np.mean(F_mit_backend), np.std(F_mit_backend)))

Graph-state fidelity estimates

No mitigation
F(ibmq_casablanca) = 0.797 ± 0.002

CTMP error mitigation
F(ibmq_casablanca) = 0.929 ± 0.003


## Qiskit version

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table